In [3]:
# Step 0. Import libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Step 1. Load data, show info and sample
# 1.1 Read data
df_raw = pd.read_excel(
   'https://github.com/datagy/mediumdata/raw/master/sample_pivot.xlsx', 
   parse_dates=['Date']
)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1000 non-null   datetime64[ns]
 1   Region  1000 non-null   object        
 2   Type    1000 non-null   object        
 3   Units   911 non-null    float64       
 4   Sales   1000 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 39.2+ KB


In [6]:
# 1.2 Show sample
df_raw.sample(5, random_state=1234)

,Date,Region,Type,Units,Sales
681,2020-11-27,East,Children's Clothing,32.0,342
990,2020-12-17,North,Children's Clothing,7.0,217
155,2020-06-13,East,Children's Clothing,30.0,336
768,2020-06-25,North,Women's Clothing,5.0,644
438,2020-10-07,North,Women's Clothing,30.0,462


In [7]:
# Step 2. Transform to standard format
# 2.1 Create a pandas chain to convert to final form
df = (
    df_raw
    .copy()
    .set_axis(
        df_raw
        .columns.str.lower()
        .str.replace(' ','_')
        .str.replace(r'\W','',regex=True)
        .str.slice(0,40), axis=1
    )
    .astype({'region':'category', 
             'type':'category'})
    .assign(
        date = lambda x: pd.to_datetime(x['date'], format='%Y-%m-%d')
    )
    .set_index('date')
)

In [8]:
# Step 3. Perform busines analysis
# 3.1 Get the summary, for categorical and numerical columns
display(df.describe(include= 'number').T)
display(df.describe(include= 'category').T)

,count,mean,std,min,25%,50%,75%,max
units,911.0,19.638858,9.471309,3.0,12.0,20.0,28.0,35.0
sales,1000.0,427.254000,253.441362,33.0,224.0,380.0,575.0,1155.0


,count,unique,top,freq
region,1000,4,East,411
type,1000,3,Women's Clothing,424


In [11]:
# 3.2 Get the sum of total sales
print(f'total sales: {df["sales"].sum()}')

total sales: 427254


In [12]:
# 3.3 Get the average of sales
print(f'Average sales: {df["sales"].mean()}')

Average sales: 427.254


In [19]:
# 3.4 Get the sum of sales by region
print("Sales by region")
sales_by_region = df.groupby("region")["sales"].sum()

print(sales_by_region)

Sales by region
region
East     167763
North    138700
South     59315
West      61476
Name: sales, dtype: int64


/tmp/ipykernel_6550/2920723695.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sales_by_region = df.groupby("region")["sales"].sum()


In [ ]:
# 3.4 Get the average sales by region
avg_sales_by_region = df.groupby("region")["sales"].mean()
print("Average Sales by region")
print(avg_sales_by_region)

Average Sales by region
region
East     408.182482
North    438.924051
South    432.956204
West     452.029412
Name: sales, dtype: float64


/tmp/ipykernel_6550/1741626865.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_sales_by_region = df.groupby("region")["sales"].mean()


In [23]:
# 3.5 Get the sum, mean and count of sales by region
count_sales_by_region = df.groupby("region")["units"].sum()

print("Data for East:")
sales_east = sales_by_region['East']
mean_east = avg_sales_by_region["East"]
count_east = count_sales_by_region["East"]
print(f'Sales for East region: {sales_east}')
print(f'Average Sales for East region: {mean_east}')
print(f'Count of Sales for East region: {count_east}')

print("Data for North:")
sales_north = sales_by_region['North']
mean_north = avg_sales_by_region["North"]
count_north = count_sales_by_region["North"]
print(f'Sales for North region: {sales_north}')
print(f'Average Sales for North region: {mean_north}')
print(f'Count of Sales for North region: {count_north}')

print("Data for South:")
sales_south = sales_by_region['South']
mean_south = avg_sales_by_region["South"]
count_south = count_sales_by_region["South"]
print(f'Sales for South region: {sales_south}')
print(f'Average Sales for South region: {mean_south}')
print(f'Count of Sales for South region: {count_south}')

print("Data for West:")
sales_west = sales_by_region['West']
mean_west = avg_sales_by_region["West"]
count_west = count_sales_by_region["West"]
print(f'Sales for West region: {sales_west}')
print(f'Average Sales for West region: {mean_west}')
print(f'Count of Sales for West region: {count_west}')

Data for East:
Sales for East region: 167763
Average Sales for East region: 408.1824817518248
Count of Sales for East region: 8110.0
Data for North:
Sales for North region: 138700
Average Sales for North region: 438.9240506329114
Count of Sales for North region: 4359.0
Data for South:
Sales for South region: 59315
Average Sales for South region: 432.95620437956205
Count of Sales for South region: 2798.0
Data for West:
Sales for West region: 61476
Average Sales for West region: 452.02941176470586
Count of Sales for West region: 2624.0


/tmp/ipykernel_6550/3215383462.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_sales_by_region = df.groupby("region")["units"].sum()


In [24]:
# 3.6 Create a pivot table that gets total units by type
pivot_table = df.pivot_table(values='units', index='type', aggfunc='sum')
print(pivot_table)

                      units
type                       
Children's Clothing  5887.0
Men's Clothing       3974.0
Women's Clothing     8030.0


/tmp/ipykernel_6550/680530046.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = df.pivot_table(values='units', index='type', aggfunc='sum')


In [25]:
# 3.7 Display the dataset showing only the East region
east_region_df = df[df['region'] == 'East']
print(east_region_df)

           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
2020-05-01   East       Men's Clothing   10.0    140
...           ...                  ...    ...    ...
2020-03-20   East       Men's Clothing   14.0    238
2020-06-05   East  Children's Clothing   11.0    220
2020-05-04   East       Men's Clothing    6.0    108
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[411 rows x 4 columns]


In [27]:
# 3.8 Display the dataset showing only the East and West regions
combined_region_df = df[(df['region'] == 'East') | (df['region'] == 'West')]
print(combined_region_df)

           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-19   West     Women's Clothing    3.0     33
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
...           ...                  ...    ...    ...
2020-02-08   West       Men's Clothing   32.0    928
2020-05-04   East       Men's Clothing    6.0    108
2020-11-17   West       Men's Clothing   27.0    486
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[547 rows x 4 columns]


In [28]:
# 3.9 Display the dataset showing East sales with units over 30 
east_high_units_df = df[(df['region'] == 'East') & (df['units'] > 30)]
print(east_high_units_df)

           region                 type  units  sales
date                                                
2020-11-03   East  Children's Clothing   34.0    374
2020-06-12   East     Women's Clothing   35.0   1050
2020-05-11   East       Men's Clothing   35.0    700
2020-06-14   East       Men's Clothing   32.0    416
2020-10-28   East       Men's Clothing   34.0    986
...           ...                  ...    ...    ...
2020-02-24   East     Women's Clothing   32.0    693
2020-08-13   East       Men's Clothing   32.0    132
2020-10-04   East       Men's Clothing   35.0    350
2020-10-20   East       Men's Clothing   32.0    928
2020-02-11   East  Children's Clothing   35.0    735

[70 rows x 4 columns]


In [29]:
# 3.10 A data corruption has been detected. Add 1 to all units. 
df["units"] = df["units"] + 1

In [31]:
df.sample(10, random_state=1234)

,region,type,units,sales
date,,,,
2020-11-27,East,Children's Clothing,33.0,342
2020-12-17,North,Children's Clothing,8.0,217
2020-06-13,East,Children's Clothing,31.0,336
2020-06-25,North,Women's Clothing,6.0,644
2020-10-07,North,Women's Clothing,31.0,462
2020-09-08,East,Women's Clothing,4.0,676
2020-04-20,East,Women's Clothing,16.0,448
2020-06-03,South,Men's Clothing,23.0,207
2020-09-24,East,Children's Clothing,26.0,310


In [32]:
# 3.11 Another issue was detected, in West region add 2 to all units
df.loc[df['region'] == 'West', 'units'] += 2